# BUILDING THE DETECTION ENGINE 

- install the following

In [ ]:
!pip install -q inference-gpu
!pip install -q trackers
!pip install -q supervision==0.27.0rc1

^C


In [ ]:
import cv2
from ultralytics import YOLO

model = YOLO("weights.pt")
tracker = SORTTracker()




def detection_engine():
    """our main goal here is to:
    - read frames from video source
    - run detection model on each frame
    - run SORT algorithm to track detected objects
    - emit processed frames to HMI via socket.emit()
    """
    cap = cv2.VideoCapture(0)  # video source
    while True:
        ret, frame = cap.read()
        if not ret:
            continue
        # load into YOLO model
        results = model(frame)
        detections = []

        for det in results[0].boxes.xyxy:  # xyxy format
            x1, y1, x2, y2 = det.cpu().numpy()
            conf = det.conf.cpu().numpy()
            detections.append([x1, y1, x2, y2, conf])
        tracks = tracker.update(detections)  # returns tracked objects with IDs
        for track in tracks:
            x1, y1, x2, y2, track_id = track
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
            cv2.putText(frame, str(track_id), (x1, y1-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)
        # here, instead of emitting directly, we would insert the frame into a thread-safe queue along with info needed by projectile prediction engine


detection_engine()

KeyboardInterrupt: 